# Description:

Imagine reading and realize in the end this article is not what you are looking for. Feels bad. Don't fret, we got you covered. Use our "TLDR Bot" to help you figure out if a article is what you needed for.

Utilize TF-IDF to rank sentences based on importance and extract "useful" text.

# Import packages 

In [16]:
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Get text from file


In [14]:
with open("testarticle.txt", "r") as file:
    text = file.read()

text = sent_tokenize(text)
print(text)
total_documents = len(text)

['For those who had academic writing, summarization — the task of producing a concise and fluent summary while preserving key information content and overall meaning — was if not a nightmare, then a constant challenge close to guesswork to detect what the professor would find important.', 'Though the basic idea looks simple: find the gist, cut off all opinions and detail, and write a couple of perfect sentences, the task inevitably ended up in toil and turmoil.', 'On the other hand, in real life we are perfect summarizers: we can describe the whole War and Peace in one word, be it “masterpiece” or “rubbish”.', 'We can read tons of news about state-of-the-art technologies and sum them up in “Musk sent Tesla to the Moon”.', 'We would expect that the computer could be even better.', 'Where humans are imperfect, artificial intelligence depraved of emotions and opinions of its own would do the job.', 'The story began in the 1950s.', 'An important research of these days introduced a method t

## Calculate Frequency Matrix for words Per Sentence

In [15]:
def create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix
create_frequency_matrix(text)

NameError: name 'PorterStemmer' is not defined

# Cleaning Summary text

# Tokenizer